# Análise Exploratória de Dados

Este notebook executa uma análise exploratória completa no dataset `Obesity.csv`, para analisar os dados antes de prepará-los para o feature engineering.



In [17]:
from __future__ import annotations

from pathlib import Path
from typing import Iterable

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


sns.set_theme(style="whitegrid", context="talk", palette="deep")
plt.rcParams.update(
    {
        "axes.titlesize": 16,
        "axes.labelsize": 13,
        "axes.titleweight": "semibold",
        "axes.labelcolor": "#2b2b2b",
        "xtick.color": "#2b2b2b",
        "ytick.color": "#2b2b2b",
        "figure.titlesize": 18,
    }
)


def detectar_raiz_projeto() -> Path:
    """Localizar a raiz do projeto, que contém o dataset."""
    caminho_atual = Path().resolve()
    for candidato in [caminho_atual, *caminho_atual.parents]:
        arquivo = candidato / "data" / "Obesity.csv"
        if arquivo.exists():
            return candidato
    raise FileNotFoundError(
        "Não foi possível localizar o arquivo 'Obesity.csv' na hierarquia de diretórios."
    )


PROJECT_ROOT = detectar_raiz_projeto()
DATA_PATH = PROJECT_ROOT / "data" / "Obesity.csv"
PLOTS_DIR = PROJECT_ROOT / "plots"

TARGET_COLUMN = "Obesity"
NUMERIC_FEATURES = [
    "Age",
    "Height",
    "Weight",
    "BMI",
    "FCVC",
    "NCP",
    "CH2O",
    "FAF",
    "TUE",
]
CATEGORICAL_FEATURES = [
    "Gender",
    "family_history",
    "FAVC",
    "CAEC",
    "SMOKE",
    "SCC",
    "CALC",
    "MTRANS",
]

LABELS_PT = {
    "Gender": "Gênero declarado",
    "Age": "Idade (anos)",
    "Height": "Altura (m)",
    "Weight": "Peso (kg)",
    "family_history": "Histórico familiar de obesidade",
    "FAVC": "Consumo frequente de alimentos calóricos",
    "FCVC": "Frequência de consumo de vegetais",
    "NCP": "Número de refeições principais por dia",
    "CAEC": "Petiscos entre refeições",
    "SMOKE": "Hábito de fumar",
    "CH2O": "Copos de água por dia",
    "SCC": "Monitoramento diário de calorias",
    "FAF": "Atividade física semanal (horas)",
    "TUE": "Tempo de uso de tecnologia (horas/dia)",
    "CALC": "Consumo de bebidas alcoólicas",
    "MTRANS": "Meio de transporte predominante",
    "Obesity": "Classificação de obesidade",
    "BMI": "Índice de Massa Corporal (IMC)",
}

EIXO_Y_PARTICIPANTES = "Número de participantes"


In [18]:
def garantir_diretorio_plots(diretorio: Path) -> None:
    """Garantir que o diretório para plots existe."""
    diretorio.mkdir(parents=True, exist_ok=True)


def carregar_dados(caminho: Path) -> pd.DataFrame:
    """Carregar o dataset no df."""
    if not caminho.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")
    return pd.read_csv(caminho)


def analise_inicial(df: pd.DataFrame) -> None:
    """Roda checks iniciais no dataset."""
    print(" ===== Informações da base =====")
    df.info()

    print(" ===== Estatísticas descritivas (numéricas) =====")
    print(df.describe())

    print(" ===== Estatísticas descritivas (categóricas) =====")
    categ = df.select_dtypes(include="object").columns
    if len(categ) > 0:
        print(df[categ].describe(include="object"))
    else:
        print("Nenhuma variável categórica encontrada.")

    print(" ===== Valores nulos por coluna =====")
    print(df.isnull().sum())

    print(" ===== Total de linhas duplicadas =====")
    print(df.duplicated().sum())


def criar_bmi(df: pd.DataFrame) -> pd.DataFrame:
    """Criar feature BMI (IMC) usando Weight e Height."""
    resultado = df.copy()
    altura = resultado["Height"].replace({0: np.nan})
    resultado["BMI"] = resultado["Weight"] / (altura**2)
    return resultado


def salvar_figura(nome_arquivo: str) -> None:
    """Salva a figura do matplotlib atual para o diretório de plots."""
    caminho = PLOTS_DIR / nome_arquivo
    plt.tight_layout()
    plt.savefig(caminho, dpi=300)
    plt.close()
    print(f"Gráfico salvo em: {caminho}")


def _obter_rotulo(coluna: str) -> str:
    """Retorna um rótulo amigável para a coluna."""
    return LABELS_PT.get(coluna, coluna.replace("_", " ").title())


def analisar_alvo(df: pd.DataFrame, coluna_alvo: str = TARGET_COLUMN) -> None:
    """Analisar a distribuição da variável alvo e salvar um gráfico pronto para apresentação."""
    if coluna_alvo not in df.columns:
        raise KeyError(f"Coluna alvo '{coluna_alvo}' não encontrada no DataFrame.")

    ordem = df[coluna_alvo].value_counts().index
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.countplot(data=df, x=coluna_alvo, order=ordem, palette="viridis", ax=ax)
    ax.set_title("Distribuição dos níveis de obesidade na população avaliada", pad=18)
    ax.set_xlabel(_obter_rotulo(coluna_alvo))
    ax.set_ylabel(EIXO_Y_PARTICIPANTES)
    ax.tick_params(axis="x", rotation=20)
    ax.set_ylim(0, df[coluna_alvo].value_counts().max() * 1.12)
    for container in ax.containers:
        ax.bar_label(container, fmt="%d", padding=3)
    sns.despine()
    salvar_figura("1_target_distribution.png")

    contagem = df[coluna_alvo].value_counts()
    percentual = contagem / len(df) * 100
    print(" ===== Distribuição do alvo =====")
    for classe, qtd in contagem.items():
        pct = percentual[classe]
        print(f"{classe}: {qtd} registros ({pct:.2f}%)")


In [19]:
def _filtrar_features_existentes(df: pd.DataFrame, features: Iterable[str]) -> list[str]:
    """Filtra nomes das features para as existentes no DF."""
    return [col for col in features if col in df.columns]


def analisar_univariada(df: pd.DataFrame) -> None:
    """Gera plots univariados com foco em comunicação executiva."""
    colunas_numericas = _filtrar_features_existentes(df, NUMERIC_FEATURES)
    for coluna in colunas_numericas:
        descricao = _obter_rotulo(coluna)
        fig, ax = plt.subplots(figsize=(9, 5))
        sns.histplot(data=df, x=coluna, kde=True, bins=30, color="#0d6efd", ax=ax)
        media = df[coluna].mean()
        mediana = df[coluna].median()
        ax.axvline(media, color="#d62728", linestyle="--", linewidth=1.5, label=f"Média: {media:.2f}")
        ax.axvline(mediana, color="#2ca02c", linestyle=":", linewidth=1.5, label=f"Mediana: {mediana:.2f}")
        ax.set_title(f"Distribuição de {descricao} na amostra")
        ax.set_xlabel(descricao)
        ax.set_ylabel(EIXO_Y_PARTICIPANTES)
        ax.legend(loc="upper right", frameon=False)
        sns.despine()
        salvar_figura(f"1_univar_{coluna.lower()}_hist.png")

    colunas_categoricas = _filtrar_features_existentes(df, CATEGORICAL_FEATURES)
    for coluna in colunas_categoricas:
        descricao = _obter_rotulo(coluna)
        ordem = df[coluna].value_counts().index
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.countplot(data=df, x=coluna, order=ordem, palette="viridis", ax=ax)
        ax.set_title(f"Distribuição de {descricao}")
        ax.set_xlabel(descricao)
        ax.set_ylabel(EIXO_Y_PARTICIPANTES)
        ax.tick_params(axis="x", rotation=25)
        for container in ax.containers:
            ax.bar_label(container, fmt="%d", padding=3)
        sns.despine()
        salvar_figura(f"1_univar_{coluna.lower()}_count.png")


def analisar_bivariada(df: pd.DataFrame, coluna_alvo: str = TARGET_COLUMN) -> None:
    """Gera plots bivariados comparando features com a target."""
    if coluna_alvo not in df.columns:
        raise KeyError(f"Coluna alvo '{coluna_alvo}' não encontrada no DataFrame.")

    ordem_alvo = df[coluna_alvo].value_counts().index
    rotulo_alvo = _obter_rotulo(coluna_alvo)

    colunas_numericas = _filtrar_features_existentes(df, NUMERIC_FEATURES)
    for coluna in colunas_numericas:
        if coluna == coluna_alvo:
            continue
        descricao = _obter_rotulo(coluna)
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.boxplot(data=df, x=coluna_alvo, y=coluna, order=ordem_alvo, palette="viridis", ax=ax)
        ax.set_title(f"{descricao} por nível de obesidade")
        ax.set_xlabel(rotulo_alvo)
        ax.set_ylabel(descricao)
        ax.tick_params(axis="x", rotation=15)
        sns.despine()
        salvar_figura(f"2_bivar_{coluna.lower()}_vs_{coluna_alvo.lower()}.png")

    colunas_categoricas = _filtrar_features_existentes(df, CATEGORICAL_FEATURES)
    for coluna in colunas_categoricas:
        descricao = _obter_rotulo(coluna)
        ordem = df[coluna].value_counts().index
        fig, ax = plt.subplots(figsize=(14, 6))
        sns.countplot(
            data=df,
            x=coluna,
            hue=coluna_alvo,
            order=ordem,
            hue_order=ordem_alvo,
            palette="viridis",
            ax=ax,
        )
        ax.set_title(f"Distribuição de {descricao} por nível de obesidade")
        ax.set_xlabel(descricao)
        ax.set_ylabel(EIXO_Y_PARTICIPANTES)
        ax.tick_params(axis="x", rotation=25)
        ax.legend(title=rotulo_alvo, bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False)
        for container in ax.containers:
            ax.bar_label(container, fmt="%d", padding=3)
        sns.despine()
        salvar_figura(f"2_bivar_{coluna.lower()}_vs_{coluna_alvo.lower()}.png")


def analisar_correlacao(df: pd.DataFrame) -> None:
    """Gera e salva um mapa de calor de correlação para as features numéricas."""
    colunas_numericas = _filtrar_features_existentes(df, NUMERIC_FEATURES)
    if not colunas_numericas:
        print("Nenhuma coluna numérica disponível para correlação.")
        return

    matriz_corr = df[colunas_numericas].corr()
    mask = np.triu(np.ones_like(matriz_corr, dtype=bool))
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.heatmap(
        matriz_corr,
        mask=mask,
        annot=True,
        fmt=".2f",
        cmap="YlGnBu",
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.8, "label": "Correlação de Pearson"},
        ax=ax,
        annot_kws={"size": 10},
    )
    ax.set_title("Correlação entre indicadores numéricos de saúde e estilo de vida", pad=18)
    ax.set_xlabel("Variáveis")
    ax.set_ylabel("Variáveis")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    sns.despine()
    salvar_figura("3_correlation_heatmap.png")


In [20]:
garantir_diretorio_plots(PLOTS_DIR)
df = carregar_dados(DATA_PATH)
analise_inicial(df)

 ===== Informações da base =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          2111 non-null   object 
 1   Age             2111 non-null   float64
 2   Height          2111 non-null   float64
 3   Weight          2111 non-null   float64
 4   family_history  2111 non-null   object 
 5   FAVC            2111 non-null   object 
 6   FCVC            2111 non-null   float64
 7   NCP             2111 non-null   float64
 8   CAEC            2111 non-null   object 
 9   SMOKE           2111 non-null   object 
 10  CH2O            2111 non-null   float64
 11  SCC             2111 non-null   object 
 12  FAF             2111 non-null   float64
 13  TUE             2111 non-null   float64
 14  CALC            2111 non-null   object 
 15  MTRANS          2111 non-null   object 
 16  Obesity         2111 non-null   object 
dtype

In [21]:
df_bmi = criar_bmi(df)
analisar_alvo(df_bmi)

C:\Users\Arnaldo Laudares\AppData\Local\Temp\ipykernel_16732\1748884178.py:64: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x=coluna_alvo, order=ordem, palette="viridis", ax=ax)


Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_target_distribution.png
 ===== Distribuição do alvo =====
Obesity_Type_I: 351 registros (16.63%)
Obesity_Type_III: 324 registros (15.35%)
Obesity_Type_II: 297 registros (14.07%)
Overweight_Level_I: 290 registros (13.74%)
Overweight_Level_II: 290 registros (13.74%)
Normal_Weight: 287 registros (13.60%)
Insufficient_Weight: 272 registros (12.88%)


In [22]:
analisar_univariada(df_bmi)
analisar_bivariada(df_bmi)
analisar_correlacao(df_bmi)

Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_age_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_height_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_weight_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_bmi_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_fcvc_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_ncp_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_ch2o_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_faf_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_tue_hist.png
Gráfico salvo em: C:\Users\Ar

C:\Users\Arnaldo Laudares\AppData\Local\Temp\ipykernel_16732\1374273347.py:29: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x=coluna, order=ordem, palette="viridis", ax=ax)
C:\Users\Arnaldo Laudares\AppData\Local\Temp\ipykernel_16732\1374273347.py:29: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x=coluna, order=ordem, palette="viridis", ax=ax)


Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_family_history_count.png


C:\Users\Arnaldo Laudares\AppData\Local\Temp\ipykernel_16732\1374273347.py:29: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x=coluna, order=ordem, palette="viridis", ax=ax)


Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_favc_count.png


...

Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_ch2o_vs_obesity.png


C:\Users\Arnaldo Laudares\AppData\Local\Temp\ipykernel_16732\1374273347.py:54: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, x=coluna_alvo, y=coluna, order=ordem_alvo, palette="viridis", ax=ax)


Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_faf_vs_obesity.png


C:\Users\Arnaldo Laudares\AppData\Local\Temp\ipykernel_16732\1374273347.py:54: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, x=coluna_alvo, y=coluna, order=ordem_alvo, palette="viridis", ax=ax)


Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_tue_vs_obesity.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_gender_vs_obesity.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_family_history_vs_obesity.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_favc_vs_obesity.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_caec_vs_obesity.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_smoke_vs_obesity.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_scc_vs_obesity.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\2_bivar_calc_vs_obesity.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\

# **Análise dos Resultados**:

* **Base limpa**: 2 111 registros, 17 colunas e nenhum valor ausente; apenas 24 linhas duplicadas, que podem ser removidas antes da modelagem para evitar viés em validações.

* **Perfil dos participantes**: média de 24 anos, altura ~1,70 m e peso 86,6 kg apontam para uma amostra de jovens adultos já com tendência a sobrepeso; desvio alto em peso (±26 kg) sugere ampla variação e possíveis outliers que merecem inspeção nos boxplots.

* **Hábitos alimentares/estilo de vida**: FCVC (~2,4) e NCP (~2,7) indicam consumo razoável de vegetais e 3 refeições diárias, porém CH2O ~2 copos e FAF ~1 (pouca atividade física) combinados com TUE <1 h mostram um grupo pouco ativo e possivelmente desidratado; esses padrões ajudam a explicar as altas taxas de obesidade.

* **Categóricas desbalanceadas**: 51 % homens, 82 % com histórico familiar de obesidade, 89 % afirmam consumo frequente de fast food e 75 % utilizam transporte público; a variabilidade concentrada em poucas categorias exige encoding cuidadoso (ex.: one-hot) e pode demandar agregações para CAEC/CALC.

* **Alvo multiclasse equilibrado o suficiente**: sete níveis e maior concentração nos três tipos de obesidade (46 %), seguidos por dois níveis de sobrepeso (~27 %) e faixas normais/baixo peso (~26 %); não há classes extremamente raras, então modelos multiclasses podem usar pesos padrão, mas monitorar métricas por classe continua essencial.

# Insights Estratégicos e Implicações para Modelagem

